
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

In [12]:
# %stop_session

# Step 1: Define your configurations

In [31]:
%connections hudi-connection
%glue_version 3.0
%region us-west-2
%worker_type G.1X
%number_of_workers 3
%spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer
%additional_python_modules Faker

You are already connected to a glueetl session 54019378-cddd-4a85-9f0d-0fb532ef92ba.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Connections to be included:
hudi-connection


You are already connected to a glueetl session 54019378-cddd-4a85-9f0d-0fb532ef92ba.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 3.0


You are already connected to a glueetl session 54019378-cddd-4a85-9f0d-0fb532ef92ba.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


...

Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 54019378-cddd-4a85-9f0d-0fb532ef92ba.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 3
Setting new number of workers to: 3
Previous Spark configuration: spark.serializer=org.apache.spark.serializer.KryoSerializer
Setting new Spark configuration to: spark.serializer=org.apache.spark.serializer.KryoSerializer


You are already connected to a glueetl session 54019378-cddd-4a85-9f0d-0fb532ef92ba.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Additional python modules to be included:
Faker


# Step 2: Define your Imports

In [81]:
try:
    import sys
    from awsglue.transforms import *
    from awsglue.utils import getResolvedOptions
    from pyspark.context import SparkContext
    from awsglue.context import GlueContext
    from awsglue.job import Job
    from pyspark.sql.session import SparkSession
    from awsglue.dynamicframe import DynamicFrame
    from pyspark.sql.functions import col, to_timestamp, monotonically_increasing_id, to_date, when
    from pyspark.sql.functions import *
    from awsglue.utils import getResolvedOptions
    from pyspark.sql.types import *
    from datetime import datetime
    import boto3
    from functools import reduce
except Exception as e:
    pass

# Step 3:  Create Spark Session

In [82]:
spark = SparkSession.builder.config('spark.serializer','org.apache.spark.serializer.KryoSerializer').config('spark.sql.hive.convertMetastoreParquet','false').config('spark.sql.legacy.pathOptionBehavior.enabled', 'true').getOrCreate()
sc = spark.sparkContext
glueContext = GlueContext(sc)
job = Job(glueContext)
logger = glueContext.get_logger()

# Step 4: Define Data Generator Class 

In [83]:
from faker import Faker
global faker
faker = Faker()

class DataGenerator(object):

    @staticmethod
    def get_data():
        return [
            (
                x,
                faker.name(),
                faker.random_element(elements=('IT', 'HR', 'Sales', 'Marketing')),
                faker.random_element(elements=('CA', 'NY', 'TX', 'FL', 'IL', 'RJ')),
                faker.random_int(min=10000, max=150000),
                faker.random_int(min=18, max=60),
                faker.random_int(min=0, max=100000),
                faker.unix_time()
            ) for x in range(3)
        ]


# Step 5: Create Spark Data frame

In [84]:
data = DataGenerator.get_data()

columns = ["emp_id", "employee_name", "department", "state", "salary", "age", "bonus", "ts"]
spark_df = spark.createDataFrame(data=data, schema=columns)

In [85]:
spark_df.show()

+------+--------------+----------+-----+------+---+-----+----------+
|emp_id| employee_name|department|state|salary|age|bonus|        ts|
+------+--------------+----------+-----+------+---+-----+----------+
|     0|Cassidy Hansen|        IT|   FL|106824| 51|25134| 631540494|
|     1|   Sherri Long|     Sales|   FL|119716| 54|50529| 465602763|
|     2|  Sandra Smith|        IT|   IL|101594| 38| 7895|1035622594|
+------+--------------+----------+-----+------+---+-----+----------+


In [19]:
spark_df = ChangeSchemaApplyMapping.toDF()


NameError: name 'ChangeSchemaApplyMapping' is not defined


In [ ]:
spark_df.show()

# Step 7:  Define your HUDI Settings 

In [86]:
db_name = "hudidb"
table_name="hudi_table"

recordkey = 'emp_id'
precombine = 'ts'

path = "s3://soumil-dms-learn/hudi/hudi_table/"
method = 'upsert'
table_type = "COPY_ON_WRITE"

In [87]:
connection_options={
    "path": path,
    "connectionName": "hudi-connection",

    "hoodie.datasource.write.storage.type": table_type,
    'className': 'org.apache.hudi',
    'hoodie.table.name': table_name,
    'hoodie.datasource.write.recordkey.field': recordkey,
    'hoodie.datasource.write.table.name': table_name,
    'hoodie.datasource.write.operation': method,
    'hoodie.datasource.write.precombine.field': precombine,


    'hoodie.datasource.hive_sync.enable': 'true',
    "hoodie.datasource.hive_sync.mode":"hms",
    'hoodie.datasource.hive_sync.sync_as_datasource': 'false',
    'hoodie.datasource.hive_sync.database': db_name,
    'hoodie.datasource.hive_sync.table': table_name,
    'hoodie.datasource.hive_sync.use_jdbc': 'false',
    'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor',
    'hoodie.datasource.write.hive_style_partitioning': 'true',
}

# Step 8: Write to HUDI 

In [88]:
ApacheHudiConnector0101forAWSGlue30_node1671045598524 = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(spark_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="glue_df",
    )
)

# Step 9 : Read from HUDI Table

In [89]:
ReadGlueDF = (
    glueContext.create_dynamic_frame.from_options(
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="ReadGlueDF",
    )
)
glue_to_spark_df = ReadGlueDF.toDF()

In [90]:
glue_to_spark_df.show()

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id| employee_name|department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+----------+-----+------+---+-----+----------+
|  20221218143859751|20221218143859751...|                 2|                      |e7ad30a4-27ce-4de...|     2|  Sandra Smith|        IT|   IL|101594| 38| 7895|1035622594|
|  20221218143859751|20221218143859751...|                 0|                      |e7ad30a4-27ce-4de...|     0|Cassidy Hansen|        IT|   FL|106824| 51|25134| 631540494|
|  20221218143859751|20221218143859751...|                 1|                      |e7ad30a4-27ce-4de...|     1|   Sherri Long|     Sal

# Step 10 Append into HUDI

In [91]:
impleDataUpd = [
    (3, "This is APPEND", "Sales", "RJ", 81000, 30, 23000, 827307999),
    (4, "This is APPEND", "Engineering", "RJ", 79000, 53, 15000, 1627694678),
]

columns = ["emp_id", "employee_name", "department", "state", "salary", "age", "bonus", "ts"]
spark_df = spark.createDataFrame(data=impleDataUpd, schema=columns)

WriteDF = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(spark_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="glue_df",
    )
)

In [92]:
ReadGlueDF = (
    glueContext.create_dynamic_frame.from_options(
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="ReadGlueDF",
    )
)
glue_to_spark_df = ReadGlueDF.toDF()
glue_to_spark_df.show()

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id| employee_name| department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|  20221218143945423|20221218143945423...|                 4|                      |e7ad30a4-27ce-4de...|     4|This is APPEND|Engineering|   RJ| 79000| 53|15000|1627694678|
|  20221218143859751|20221218143859751...|                 2|                      |e7ad30a4-27ce-4de...|     2|  Sandra Smith|         IT|   IL|101594| 38| 7895|1035622594|
|  20221218143945423|20221218143945423...|                 3|                      |e7ad30a4-27ce-4de...|     3|This is APPEND|   

# Step 11: Update in HUDI table

In [93]:
impleDataUpd = [
    (3, "this is update on data lake", "Sales", "RJ", 81000, 30, 23000, 827307999),
]
columns = ["emp_id", "employee_name", "department", "state", "salary", "age", "bonus", "ts"]
spark_df = spark.createDataFrame(data=impleDataUpd, schema=columns)

WriteDF = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(spark_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="glue_df",
    )
)

ReadGlueDF = (
    glueContext.create_dynamic_frame.from_options(
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="ReadGlueDF",
    )
)
glue_to_spark_df = ReadGlueDF.toDF()
glue_to_spark_df.show()


+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id|       employee_name| department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+-----------+-----+------+---+-----+----------+
|  20221218143945423|20221218143945423...|                 4|                      |e7ad30a4-27ce-4de...|     4|      This is APPEND|Engineering|   RJ| 79000| 53|15000|1627694678|
|  20221218143859751|20221218143859751...|                 2|                      |e7ad30a4-27ce-4de...|     2|        Sandra Smith|         IT|   IL|101594| 38| 7895|1035622594|
|  20221218144030080|20221218144030080...|                 3|                      |e7ad30a4-27ce-4d

# Step 12: Query with Spark SQL

In [94]:
spark.sql("select * from  hudidb.hudi_table where emp_id = 3 ").show()

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+----------+-----+------+---+-----+---------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id|       employee_name|department|state|salary|age|bonus|       ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+----------+-----+------+---+-----+---------+
|  20221218144030080|20221218144030080...|                 3|                      |e7ad30a4-27ce-4de...|     3|this is update on...|     Sales|   RJ| 81000| 30|23000|827307999|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+----------+-----+------+---+-----+---------+


# Step 13: Hard Delete

In [95]:
spark_df = spark.sql("SELECT * FROM hudidb.hudi_table where emp_id='4' ")

print("********************")
print(spark_df.show())
print("********************")

db_name = "hudidb"
table_name="hudi_table"

recordkey = 'emp_id'
precombine = 'ts'

path = "s3://soumil-dms-learn/hudi/hudi_table/"
method = 'delete'
table_type = "COPY_ON_WRITE"

connection_options={
    "path": path,
    "connectionName": "hudi-connection",

    "hoodie.datasource.write.storage.type": table_type,
    'className': 'org.apache.hudi',
    'hoodie.table.name': table_name,
    'hoodie.datasource.write.recordkey.field': recordkey,
    'hoodie.datasource.write.table.name': table_name,
    'hoodie.datasource.write.operation': method,
    'hoodie.datasource.write.precombine.field': precombine,


    'hoodie.datasource.hive_sync.enable': 'true',
    "hoodie.datasource.hive_sync.mode":"hms",
    'hoodie.datasource.hive_sync.sync_as_datasource': 'false',
    'hoodie.datasource.hive_sync.database': db_name,
    'hoodie.datasource.hive_sync.table': table_name,
    'hoodie.datasource.hive_sync.use_jdbc': 'false',
    'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor',
    'hoodie.datasource.write.hive_style_partitioning': 'true',
}



WriteDF = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(spark_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="glue_df",
    )
)

ReadGlueDF = (
    glueContext.create_dynamic_frame.from_options(
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="ReadGlueDF",
    )
)
glue_to_spark_df = ReadGlueDF.toDF()
glue_to_spark_df.show()

********************
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id| employee_name| department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|  20221218143945423|20221218143945423...|                 4|                      |e7ad30a4-27ce-4de...|     4|This is APPEND|Engineering|   RJ| 79000| 53|15000|1627694678|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+

None
********************
+-------------------+--------------------+------------------+---------------------

# Step 14: Clustering and compaction

 ![image](https://user-images.githubusercontent.com/39345855/208302373-6624c2fc-b2ed-4fee-bb60-984d7cb0ec20.png)


In [97]:
db_name = "hudidb"
table_name="hudi_table"
recordkey = 'emp_id'
precombine = 'ts'
path = "s3://soumil-dms-learn/hudi/hudi_table/"
method = 'upsert'
table_type = "COPY_ON_WRITE"

connection_options={
    "path": path,
    "connectionName": "hudi-connection",

    "hoodie.datasource.write.storage.type": table_type,
    'className': 'org.apache.hudi',
    'hoodie.table.name': table_name,
    'hoodie.datasource.write.recordkey.field': recordkey,
    'hoodie.datasource.write.table.name': table_name,
    'hoodie.datasource.write.operation': method,
    'hoodie.datasource.write.precombine.field': precombine,


    'hoodie.datasource.hive_sync.enable': 'true',
    "hoodie.datasource.hive_sync.mode":"hms",
    'hoodie.datasource.hive_sync.sync_as_datasource': 'false',
    'hoodie.datasource.hive_sync.database': db_name,
    'hoodie.datasource.hive_sync.table': table_name,
    'hoodie.datasource.hive_sync.use_jdbc': 'false',
    'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor',
    'hoodie.datasource.write.hive_style_partitioning': 'true',
    
    "hoodie.clustering.plan.strategy.sort.columns":"state",
    "hoodie.clustering.plan.strategy.max.bytes.per.group" : '107374182400',
    'hoodie.clustering.plan.strategy.max.num.groups' : '1',
    'hoodie.cleaner.policy' : 'KEEP_LATEST_FILE_VERSIONS',
    
}


In [98]:
cluster_df = spark.sql("SELECT * FROM hudidb.hudi_table ")

WriteDF = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(cluster_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="glue_df",
    )
)

 ![image](https://user-images.githubusercontent.com/39345855/208302689-cd37ea24-6b65-411d-bc7f-146a3f979c7a.png)


# Step 15: Time Travel Query

In [99]:
df = spark.read. \
  format("hudi"). \
  option("as.of.instant", "2022-18-12"). \
  load(path)

In [100]:
df.show()

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id|       employee_name|department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+----------+-----+------+---+-----+----------+
|  20221218144316383|20221218144316383...|                 1|                      |e7ad30a4-27ce-4de...|     1|         Sherri Long|     Sales|   FL|119716| 54|50529| 465602763|
|  20221218144316383|20221218144316383...|                 0|                      |e7ad30a4-27ce-4de...|     0|      Cassidy Hansen|        IT|   FL|106824| 51|25134| 631540494|
|  20221218144316383|20221218144316383...|                 2|                      |e7ad30a4-27ce-4de...|

# Incremental query

In [101]:
spark. \
      read. \
      format("hudi"). \
      load(path). \
      createOrReplaceTempView("hudi_snapshot")

In [102]:
commits = list(map(lambda row: row[0], spark.sql("select distinct(_hoodie_commit_time) as commitTime from  hudi_snapshot order by commitTime").limit(50).collect()))
beginTime = commits[len(commits) - 2] # commit time we are interested in

In [103]:
beginTime

'20221218144316383'


In [104]:
commits

['20221218144316383']


In [105]:
incremental_read_options = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': beginTime,
}

In [106]:
IncrementalDF = spark.read.format("hudi"). \
  options(**incremental_read_options). \
  load(path)

IncrementalDF.createOrReplaceTempView("hudi_incremental")

In [107]:
spark.sql("select * from  hudi_incremental").show()


+-------------------+--------------------+------------------+----------------------+-----------------+------+-------------+----------+-----+------+---+-----+---+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name|emp_id|employee_name|department|state|salary|age|bonus| ts|
+-------------------+--------------------+------------------+----------------------+-----------------+------+-------------+----------+-----+------+---+-----+---+
+-------------------+--------------------+------------------+----------------------+-----------------+------+-------------+----------+-----+------+---+-----+---+


# Appending data for incremental data processing 

In [108]:
impleDataUpd = [
    (6, "This is APPEND", "Sales", "RJ", 81000, 30, 23000, 827307999),
    (7, "This is APPEND", "Engineering", "RJ", 79000, 53, 15000, 1627694678),
]

columns = ["emp_id", "employee_name", "department", "state", "salary", "age", "bonus", "ts"]
spark_df = spark.createDataFrame(data=impleDataUpd, schema=columns)

WriteDF = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(spark_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="glue_df",
    )
)

In [109]:
incremental_read_options = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': beginTime,
}

In [110]:
IncrementalDF = spark.read.format("hudi"). \
  options(**incremental_read_options). \
  load(path)

IncrementalDF.createOrReplaceTempView("hudi_incremental")

In [111]:
spark.sql("select * from  hudi_incremental").show()

+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id| employee_name| department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+-----------+-----+------+---+-----+----------+
|  20221218144431412|20221218144431412...|                 6|                      |e7ad30a4-27ce-4de...|     6|This is APPEND|      Sales|   RJ| 81000| 30|23000| 827307999|
|  20221218144431412|20221218144431412...|                 7|                      |e7ad30a4-27ce-4de...|     7|This is APPEND|Engineering|   RJ| 79000| 53|15000|1627694678|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------+---